In [89]:
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import torch



True


In [3]:
class Point:
    
    """
    An instance of this class contains a latitude and a longitude.
    """
    
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude
    
    def __str__(self):
        return "({}, {})".format(self.latitude, self.longitude)

# SAMPLE USAGE

# point = Point(34.6, 78.6)
# print(point)

In [4]:
class Rectangular_Region:
    
    """
    This class defines a rectangular region. It takes two points to completely define
    a rectangular region.  
    
    Member functions : 
    
    point1 : The first point is the one with the lower value for both laittude and 
    longitude.
    
    point2 : The second point is the one with the higher value for both.
    
    height : difference in latitude values.
    width : difference in longitude values.
    """
    
    def __init__(self, point1, point2):
        
        assert(point1.latitude < point2.latitude and point1.longitude < point2.longitude)
        
        self.point1 = point1
        self.point2 = point2
        
        self.height = self.point2.latitude - self.point1.latitude
        self.width = self.point2.longitude - self.point1.longitude
        
    
    def __str__(self):
        
        return_str = "Region bounded between Latitudes ({}, {}); Longitudes ({}, {})".format(self.point1.latitude, self.point2.latitude, self.point1.longitude, self.point2.longitude)
        return return_str
    
    """This function takes a point and tells if said point lies inside the
        current rectangular region.
    """
    
    def contains(self, point):
        
        point_lat = point.latitude
        point_lon = point.longitude

        h_bounded = point.longitude >= self.point1.longitude and point.longitude <= self.point2.longitude
        v_bounded = point.latitude >= self.point1.latitude and point.latitude <= self.point2.latitude
        
        return h_bounded and v_bounded

    
# SAMPLE USAGE

# point1 = Point(34.6, 78.9)
# point2 = Point(56.5, 89.6)

# point3 = Point(35, 80)
# point4 = Point(30, 75)

# region = Rectangular_Region(point1, point2)
# print(region)
# print(region.height)
# print(region.width)
# print(region.contains(point3))
# print(region.contains(point4)) 



In [5]:
class Spatial_Segmentation:
    
    """
    This class handles the process of dividing a region into subregions.
    """
    
    def __init__(self, region, rows, cols):
        
        self.subregion_height = region.height / rows
        self.subregion_width = region.width / cols
                
        self.origin = region.point1
        self.end = region.point2
        
        
        self.origin_lat, self.origin_lon = (self.origin.latitude, self.origin.longitude)
        self.end_lat, self.end_lon = (self.end.latitude, self.end.longitude)
        
        
        subregions = []
        
        for i in range(rows):
            
            row = []
            bottom = self.origin_lat + self.subregion_height * i
            top = bottom + self.subregion_height
            
            
            for j in range(cols):
                
                left = self.origin_lon + self.subregion_width * j
                right = left + self.subregion_width
                
                
                sub_origin = Point(bottom, left)
                sub_end = Point(top, right)
                subregion = Rectangular_Region(sub_origin, sub_end)
                
                row.append(subregion)
            
            subregions.append(row)
            
            self.matrix = subregions
            self.rows = rows
            self.cols = cols
            
    def __iter__(self):
        
        for i in xrange(self.rows):
            for j in xrange(self.cols):
                yield self.matrix[i][j]  
    
    def __getitem__(self, i):
        return self.matrix[i]
    
    def __str__(self):
        
        return_str = "Listing {}*{} subregions:\n".format(self.rows, self.cols)
        
        for i in range(self.rows):
            for j in range(self.cols):
                return_str += str(self.matrix[i][j]) + "\n"
 
        return return_str

                
# SAMPLE USAGE

# print(region)

# subregions = Spatial_Segmentation(region, 3, 4)
# print(subregions)
# print(subregions[2][3]) 
# print(subregions.origin)
# print(subregions.end)  

In [6]:
class Interval:
    
    def __init__(self, start_time, end_time):
        assert(isinstance(start_time, datetime) and isinstance(end_time, datetime))
        self.start = start_time
        self.end = end_time
        
    def __str__(self):
        return "({}, {})".format(self.start, self.end)
    
    def contains(self, timestamp):
        assert(isinstance(timestamp, datetime))
        
        return self.start <= timestamp and self.end >= timestamp

#SAMPLE USAGE

# time_x = datetime(2020, 2, 10, 17, 15, 20)
# time_y = datetime(2021, 4, 11, 19, 45, 8)
# interval = Interval(time_x, time_y)
# print(interval)
# print(isinstance(interval, Interval))

# time1 = datetime(2019, 4, 5)
# time2 = datetime(2020, 3, 11)

# print(interval.contains(time1))
# print(interval.contains(time2))    
    

In [7]:
class Temporal_Segmentation:
    
    def __init__(self, time_window, num_segments):
        
        assert(isinstance(time_window, Interval))
        self.num_segments = num_segments
        self.segment_length = (time_window.end - time_window.start) / num_segments
        
        segments = []
        
        self.begin = time_window.start
        self.end = time_window.end
        
        for i in range(self.num_segments):
            start = time_window.start + i * self.segment_length
            end = start + self.segment_length
            
            segment = Interval(start, end)
            segments.append(segment)
            
        self.matrix = segments
        
    def __str__(self):
        
        return_str = ""

        for segment in self.matrix:
            return_str += str(segment) + "\n"
    
        return return_str
    
    
    def __iter__(self):
        
        for i in xrange(self.num_segments):
            yield self.matrix[i]
    
    def __getitem__(self, i):
        
        return self.matrix[i]
    
        
# SAMPLE USAGE

# time_x = datetime(2020, 2, 10, 17, 15, 20)
# time_y = datetime(2021, 4, 11, 19, 45, 8)
# interval = Interval(time_x, time_y)
# print(interval)
# print()
# subintervals = Temporal_Segmentation(interval, 4)
# print(subintervals)
        

        

In [8]:
class Earthquake:
    
    """
    This class defines the concrete structure of an Earthquake, encoding all relevant event-related data
    and providing methods for discerning additional information, like the particular space and time segment 
    a given event falls into.
    """
    
    def __init__(self, timestamp, location, magnitude):
        
        assert(isinstance(timestamp, datetime))
        assert(isinstance(location, Point))
        
        self.timestamp = timestamp
        self.location = location
        self.magnitude = magnitude
        self.to_include = True
     
    def __str__(self):
        
        return_str = "Time : {}, Location : ({}, {}), Magnitude: {}".format(self.timestamp, self.location.latitude, self.location.longitude, self.magnitude)
        
        return return_str
    
    def time_segment(self, intervals):
        
        index = int((self.timestamp - intervals[0].start).total_seconds() / intervals.segment_length.total_seconds())
        assert(intervals[index].contains(self.timestamp))
        return index
        
    def space_segment(self, subregions):
        
        assert(isinstance(subregions, Spatial_Segmentation))
        
        latitude = self.location.latitude
        longitude = self.location.longitude
        
        origin_lat = subregions.origin.latitude
        origin_long = subregions.origin.longitude
        
        i = int((latitude - origin_lat) / subregions.subregion_height)
        j = int((longitude - origin_long) / subregions.subregion_width)
        
        assert(subregions[i][j].contains(self.location))
        
        return (i, j)
        


# SAMPLE USAGE

# timestamp = datetime(2020, 2, 26)
# location = Point(56.6, 45.5)
# magnitude = 6.0
# earthquake = Earthquake(timestamp, location, magnitude)
# print(earthquake)

# time1 = datetime(2019, 3, 4)
# time2 = datetime(2021, 4, 3)

# interval = Interval(time1, time2)
# subintervals = Temporal_Segmentation(interval, 10)

# point1 = Point(23, 23)
# point2 = Point(70, 70)
# region = Rectangular_Region(point1, point2)
# subregions = Spatial_Segmentation(region, 2, 3)

# print(subintervals)
# print(earthquake.time_segment(subintervals))
# print()
# print(subregions)
# print(earthquake.space_segment(subregions))

        

In [9]:
def time_from_string_usgs(time_str):
    
    tokens = time_str.split("T")
    
    date = tokens[0]
    time = tokens[1]
    
    tokens = date.split("-")
    year = int(tokens[0])
    month = int(tokens[1])
    day = int(tokens[2])
    
    tokens = time.split(":")
    hour = int(tokens[0])
    minute = int(tokens[1])
    second = int(tokens[2][:2])
    
    return datetime(year, month, day, hour, minute, second)

# print("reload successful")

# SAMPLE USAGE

# print(time_from_string_usgs("1950-02-26T03:35:46.000Z"))
# print(isinstance(time_from_string_usgs("1950-02-26T03:35:46.000Z"), datetime))

In [10]:
def time_from_string_dmy(time_str):
    
    tokens = time_str.split("/")
    
    month = int(tokens[0])
    day = int(tokens[1])
    year = int(tokens[2])
    
    return datetime(year, month, day)

In [11]:
def extract_from_csv(file, time_parser_function, mag_range=None):
    
    df = pd.read_csv(file)
    df = df[["time", "latitude", "longitude", "mag"]]
    
    earthquakes = []
    
    for i in range(len(df)):
        
        time = df["time"][i]
        latitude = df["latitude"][i]
        longitude = df["longitude"][i]
        magnitude = df["mag"][i]
        
        time = time_parser_function(time)
        latitude = float(latitude)
        longitude = float(longitude)
        point = Point(latitude, longitude)
        magnitude = float(magnitude)      
        
        earthquake = Earthquake(time, point, magnitude)
        
        if mag_range:
            if magnitude < mag_range[0] or magnitude > mag_range[1]:
                earthquake.to_include = False
    
        earthquakes.append(earthquake)
    
    earthquakes.sort(key = lambda x : x.timestamp)
    return earthquakes

        
# SAMPLE USAGE

# earthquakes = extract_from_csv("indonesia.csv", time_from_string_dmy)
# print(earthquakes[8])

In [41]:
class Cell:
    
    """This class captures the fundamental building block part of a spatio-temporal segmentation of the study area 
    and timeline.
    """
    
    def __init__(self, subregion, subinterval):
        
        self.earthquakes = []
        self.subregion = subregion
        self.subinterval = subinterval
        
        self.freq = 0
        self.max_magnitude = 0
        self.min_magnitude = 20
        self.has_event = 0
        
    def increment_frequency(self):
        
        self.freq = self.freq + 1
        self.has_event = 1
        return
    
    def update_maxmin(self, magnitude):
        
        if(self.max_magnitude < magnitude):
            self.max_magnitude = magnitude
        
        if(self.min_magnitude > magnitude):
            self.min_magnitude = magnitude
        
        return 
    
    def __str__(self):
        
        if self.freq == 0:
            return "No earthquakes were recorded in this cell."
        
        return_str = ""
        return_str += "This cell represents the following spatio-temporal segment:\n"
        return_str += str(self.subinterval) + "\n"
        return_str += str(self.subregion) + "\n"
        return_str += "Earthquakes:\n"
        
        for earthquake in self.earthquakes:
            return_str += str(earthquake) + "\n"
        
        return_str += "Total: {}\n".format(self.freq)
        return_str += "Maximum magnitude: {}\n".format(self.max_magnitude)
        return_str += "Minimum magnitude: {}\n".format(self.min_magnitude)
        return return_str

In [56]:
class Spatio_Temporal_Segmentation:
    
    def __init__(self, rows, cols, num_intervals, earthquakes):
        
        origin_lat = min([earthquake.location.latitude for earthquake in earthquakes]) - 0.1
        origin_lon = min([earthquake.location.longitude for earthquake in earthquakes]) - 0.1
        end_lat = max([earthquake.location.latitude for earthquake in earthquakes]) + 0.1
        end_lon = max([earthquake.location.longitude for earthquake in earthquakes]) + 0.1
        
        start = min([earthquake.timestamp for earthquake in earthquakes]) - timedelta(seconds=1)
        end = max([earthquake.timestamp for earthquake in earthquakes]) + timedelta(seconds=1)
        
        point1 = Point(origin_lat, origin_lon)
        point2 = Point(end_lat, end_lon)
        
        region = Rectangular_Region(point1, point2)
        timeline = Interval(start, end)
        
        self.region = region
        self.timeline = timeline
        self.subregions = Spatial_Segmentation(region, rows, cols)
        self.subintervals = Temporal_Segmentation(timeline, num_intervals)
        self.rows = rows
        self.cols = cols
        self.num_intervals = num_intervals
        
            
        cells = np.empty((num_intervals, rows, cols), dtype=Cell)
        
        for i in range(num_intervals):
            for j in range(rows):
                for k in range(cols):
                    cells[i][j][k] = Cell(self.subintervals[i], self.subregions[j][k])
        
        for earthquake in earthquakes:
            
            if not earthquake.to_include:
                continue
            
            time_segment = earthquake.time_segment(self.subintervals)
            row, col = earthquake.space_segment(self.subregions)
            
            cells[time_segment][row][col].earthquakes.append(earthquake)
            cells[time_segment][row][col].increment_frequency()
            cells[time_segment][row][col].update_maxmin(earthquake.magnitude)
            
        self.cells = cells
        
        
    def __iter__(self):
        
        for i in range(self.num_intervals):
            for j in range(self.rows):
                for k in range(self.cols):
                    yield self.cells[i][j][k]
    
    def __getitem__(self, i):
        
        return self.cells[i]
        
    
# SAMPLE USAGE
        
# region = Rectangular_Region(Point(-10-0.1, 115-0.1), Point(5+0.1, 130+0.1))
# timeline = Interval(datetime(1963, 2, 5), datetime(2019, 12, 30))
# rows = 4
# cols = 5
# num_intervals = 10
# earthquakes = extract_from_csv("indonesia.csv", time_from_string_dmy)

# subintervals = Temporal_Segmentation(timeline, num_intervals)
# earthquake = earthquakes[0]
# print(earthquake)
# print(subintervals[0])
# print(earthquake.time_segment(subintervals))

# subregions = Spatial_Segmentation(region, 4, 5)
# print(earthquake.location)
# print(subregions[2][2])
# print(earthquake.space_segment(subregions))
# cells = Spatio_Temporal_Segmentation(rows, cols, num_intervals, earthquakes)
# print(cells[0][0][0].max_magnitude)


# earthquakes = extract_from_csv("indonesia.csv", time_from_string_dmy)
# cells = Spatio_Temporal_Segmentation(4, 5, 100, earthquakes)
# print(cells[9][0][0].has_event)
        

In [73]:
class TensorGen:
    
    """
    Tensor data generator
    """
    
    def __init__(self, cells):
        
        self.cells = cells
        self.seq_len, self.rows, self.cols = cells.num_intervals, cells.rows, cells.cols
        
    def eventNoEvent(self):
            
        seqtensor = torch.zeros(self.seq_len, 1, self.rows * self.cols)
            
        for i in range(self.seq_len):
            for j in range(self.rows):
                for k in range(self.cols):
                    if self.cells[i][j][k].has_event:
                        seqtensor[i, 0, j*self.cols + k] = 1
                    else:
                        seqtensor[i, 0, j*self.cols + k] = 0
        
#         seqtensor = torch.LongTensor(seqtensor)
        
        return seqtensor
    
    def freq(self):
            
        seqtensor = torch.zeros(self.seq_len, 1, self.rows * self.cols)
            
        for i in range(self.seq_len):
            for j in range(self.rows):
                for k in range(self.cols):
                    seqtensor[i, 0, j*self.cols + k] = self.cells[i][j][k].freq
        return seqtensor
    
    def min_magnitude(self):
            
        seqtensor = torch.zeros(self.seq_len, 1, self.rows * self.cols)
            
        for i in range(self.seq_len):
            for j in range(self.rows):
                for k in range(self.cols):
                    seqtensor[i, 0, j*self.cols + k] = self.cells[i][j][k].min_magnitude
        return seqtensor
    
    def max_magnitude(self):
            
        seqtensor = torch.zeros(self.seq_len, 1, self.rows * self.cols)
            
        for i in range(self.seq_len):
            for j in range(self.rows):
                for k in range(self.cols):
                    seqtensor[i, 0, j*self.cols + k] = self.cells[i][j][k].max_magnitude
        return seqtensor
            
# SAMPLE USAGE

# generator = TensorGen(cells)
# freqtensor = generator.freq()

# print([cells[0][j][k].freq for j in range(rows) for k in range(cols)])

# print(freqtensor[0])

In [82]:
# earthquakes = extract_from_csv("indonesia.csv", time_from_string_dmy)
# cells = Spatio_Temporal_Segmentation(4, 5, 50, earthquakes)
# generator = TensorGen(cells)
# freqtensor = generator.freq()
# print(freqtensor.size())

torch.Size([50, 1, 20])


In [83]:
# print(cells[0][1][3])

This cell represents the following spatio-temporal segment:
Region bounded between Latitudes (-6.3, -2.5); Longitudes (124.024, 127.062)
(1963-02-05 23:59:59, 1964-03-27 14:23:59.040000)
Earthquakes:
Time : 1963-10-14 00:00:00, Location : (-3.3, 126.7), Magnitude: 4.7
Time : 1964-03-22 00:00:00, Location : (-2.7, 126.4), Magnitude: 5.1
Total: 2
Maximum magnitude: 5.1
Minimum magnitude: 4.7

